# CS591S1 - Differential Privacy - Spring 2021

## Assignment 2

This notebook provides the solution to Exercise 2 of Assignment #2. This includes the algorithms for the Report Noisy Max version of the Median Algorithm and 3 tables recording some statistics of the experiment on each of the 3 settings.

Importing some useful libraries.

In [1]:
import numpy as np
import pandas as pd
from numpy import linalg as LA
import random
import matplotlib.pyplot as plt

Method initialising the dataset for a corresponding setting.

In [2]:
def init(n, R, setting, k):
    li = []
    for i in range(n):
        if setting == 1:
            li.append(np.random.normal(R/4, (R*R)/10))
        elif setting == 2:
            li.append(np.random.poisson(50))
        elif setting == 3:
            li.append(np.random.uniform((R/2 - k), (R/2 + k)))
    return np.array(li)

Method that rounds the data values to the nearest integer in {1,...,R}.

In [3]:
def round_to_nearest(x, R):
    li = []
    for i in range(len(x)):
        if x[i] <= 0:
            li.append(1)
        elif x[i] > R:
            li.append(R)
        else:
            li.append(round(x[i]))
    return np.array(li)

Method for determining the rank of y in sorted x

In [4]:
# determine the position of y if it is inserted into the data array x
def rank(y, x):
    # using the idea of binary search for speed
    left = 0
    right = len(x)
    mid = 0
    
    while (left < right):
        mid = (right + left)//2
  
        # Check if key is present in array
        if (x[mid] == y):
            # If duplicates are present, consider the position of last duplicate
            while (mid + 1 < len(x) and x[mid + 1] == y):
                 mid += 1 # make the rank be the position of last duplicate + 1
            break
         
        # If key is smaller, ignore right half
        elif (x[mid] > y):
            right = mid
  
        # If key is greater, ignore left half
        else:
            left = mid + 1
      
    # If key is not found in array then it will be before mid
    while (mid > -1 and  x[mid] > y):
        mid-= 1
  
    return mid + 1

In [5]:
# an example
x = [1, 2, 3, 5, 6]
print(rank(5, x))

4


Implementing the sign and score functions.

In [6]:
def sign(z):
    if z > 0:
        return 1
    elif z < 0:
        return -1
    else:
        return 0

def score_func(y, x): # with the assumption that x is sorted and entries are unique
    rank_y = rank(y, x)
    n = len(x)
    abs_q = 2*(abs(rank_y - n/2)) # from problem 1b
    return -abs_q

Implementing the Report Noisy Max Algorithm.

In [7]:
def rnm_median(x, R, epsilon):
    li = []
    for y in range(1, R+1):
        Z_y = np.random.exponential(2/epsilon)
        q = score_func(y, x)
        noisy_q = q + Z_y
        li.append(noisy_q)
    return li.index(max(li)) + 1

Method for running the experiment on the first two settings of Normal and Poisson distribution.

In [8]:
def experiment(setting, n_list, R_list):
    output_data = [] # the 2D array that stores the record of each dataset
    table_data = [] # the 2D array that stores the entries of the table
    for n in n_list:
        for R in R_list:
            for i in range(0, 50): # running the experiment on each dataset
                x = init(n, R, setting, 0)
                rounded_x = round_to_nearest(x, R)
                rounded_sorted_x = np.sort(rounded_x)
                datum = [] # storing the values of the 10 runs
                for j in range(0, 10):
                    y_hat = rnm_median(rounded_sorted_x, R, 0.1)
                    datum.append(abs(n/2 - rank(y_hat, rounded_sorted_x)))
                output_data.append(datum)
            
            # adding entries to the table
            table_datum = [] # storing one row of the table
            flatten_data = np.array(output_data).flatten()
            table_datum.append(np.mean(flatten_data))
            table_datum.append(np.std(flatten_data))
            
            # storing the std of each dataset for averaging later
            stds = [] 
            for k in range(len(output_data)):
                stds.append(np.std(output_data[k]))
            table_datum.append(np.mean(stds))
            table_data.append(table_datum)
            table_datum = []
            output_data = []
            
    return table_data

Experimenting on the first setting of Normal distribution.

In [9]:
n_list = [50, 100, 500, 2000, 10000]
R_list = [100, 1000, 10000]
data_s1 = experiment(1, n_list, R_list)
df_s1 = pd.DataFrame(np.array(data_s1), 
                     columns = ['Avg error', 'Std of error', 'Std among runs'],
                     index = ['n = 50, R = 100', 'n = 50, R = 1000', 'n = 50, R = 10000',
                              'n = 100, R = 100', 'n = 100, R = 1000', 'n = 100, R = 10000',
                              'n = 500, R = 100', 'n = 500, R = 1000', 'n = 500, R = 10000',
                              'n = 2000, R = 100', 'n = 2000, R = 1000', 'n = 2000, R = 10000',
                              'n = 10000, R = 100', 'n = 10000, R = 1000', 'n = 10000, R = 10000'])
print(df_s1)

                      Avg error  Std of error  Std among runs
n = 50, R = 100           2.628      2.169243        0.616652
n = 50, R = 1000          3.070      2.266517        0.078963
n = 50, R = 10000         3.060      2.023957        0.000000
n = 100, R = 100          3.188      2.364034        1.067336
n = 100, R = 1000         3.096      2.784741        0.130360
n = 100, R = 10000        4.516      3.132690        0.019165
n = 500, R = 100          8.264      6.409236        4.231917
n = 500, R = 1000         9.204      6.654501        0.554734
n = 500, R = 10000       10.678      6.841514        0.069192
n = 2000, R = 100        11.028     11.552974        7.570988
n = 2000, R = 1000       17.506     14.041580        1.945152
n = 2000, R = 10000      16.334     12.923716        0.208978
n = 10000, R = 100       10.296     10.553122        8.998094
n = 10000, R = 1000      32.286     29.647938        6.705149
n = 10000, R = 10000     38.040     32.274609        1.139831


Experimenting on the second setting of Poisson distribution.

In [10]:
n_list = [50, 100, 500, 2000, 10000]
R_list = [100, 1000, 10000]
data_s2 = experiment(2, n_list, R_list)
df_s2 = pd.DataFrame(np.array(data_s2), 
                     columns = ['Avg error', 'Std of error', 'Std among runs'],
                     index = ['n = 50, R = 100', 'n = 50, R = 1000', 'n = 50, R = 10000',
                              'n = 100, R = 100', 'n = 100, R = 1000', 'n = 100, R = 10000',
                              'n = 500, R = 100', 'n = 500, R = 1000', 'n = 500, R = 10000',
                              'n = 2000, R = 100', 'n = 2000, R = 1000', 'n = 2000, R = 10000',
                              'n = 10000, R = 100', 'n = 10000, R = 1000', 'n = 10000, R = 10000'])
print(df_s2)

                      Avg error  Std of error  Std among runs
n = 50, R = 100          16.054      9.429055        8.681054
n = 50, R = 1000         23.534      4.913740        3.294764
n = 50, R = 10000        24.812      1.903853        0.564000
n = 100, R = 100         13.870     14.980290       13.624419
n = 100, R = 1000        35.876     19.599302       17.917115
n = 100, R = 10000       48.090      8.412960        4.732501
n = 500, R = 100         10.146      8.542405        6.191452
n = 500, R = 1000        10.386      8.785272        5.867866
n = 500, R = 10000        8.768      7.774714        5.521747
n = 2000, R = 100        35.178     16.001947        1.023733
n = 2000, R = 1000       29.842     18.430655        1.280529
n = 2000, R = 10000      35.354     14.048369        1.346605
n = 10000, R = 100      188.564     46.779204        0.088000
n = 10000, R = 1000     191.622     47.673673        0.064156
n = 10000, R = 10000    190.542     53.812454        0.398000


Method for running the experiment of the last setting. Pretty much the same as the previous experiment with a couple slight changes in values to reflect the Bimodal distribution.

In [11]:
def experiment_setting3(setting, n_list, k_list):
    output_data = [] # the 2D array that stores the record of each dataset
    table_data = [] # the 2D array that stores the entries of the table
    for n in n_list:
        for k in k_list:
            for i in range(0, 50): # running the experiment on each dataset
                x = init(n, 1000, setting, k)
                rounded_x = round_to_nearest(x, 1000)
                rounded_sorted_x = np.sort(rounded_x)
                datum = [] # storing the values of the 10 runs
                for j in range(0, 10):
                    y_hat = rnm_median(rounded_sorted_x, 1000, 0.1)
                    datum.append(abs(n/2 - rank(y_hat, rounded_sorted_x)))
                output_data.append(datum)
            
            # adding entries to the table
            table_datum = [] # storing one row of the table
            flatten_data = np.array(output_data).flatten()
            table_datum.append(np.mean(flatten_data))
            table_datum.append(np.std(flatten_data))
            
            # storing the std of each dataset for averaging later
            stds = [] 
            for u in range(len(output_data)):
                stds.append(np.std(output_data[u]))
            table_datum.append(np.mean(stds))
            table_data.append(table_datum)
            table_datum = []
            output_data = []
            
    return table_data

Experimenting on the third setting of Bimodal distribution.

In [12]:
n_list = [50, 100, 500, 2000, 10000]
k_list = [10, 100, 200]
data_s3 = experiment_setting3(3, n_list, k_list)
df_s3 = pd.DataFrame(np.array(data_s3), 
                     columns = ['Avg error', 'Std of error', 'Std among runs'],
                     index = ['n = 50, k = 10', 'n = 50, k = 100', 'n = 50, k = 200',
                              'n = 100, k = 10', 'n = 100, k = 100', 'n = 100, k = 200',
                              'n = 500, k = 10', 'n = 500, k = 100', 'n = 500, k = 200',
                              'n = 2000, k = 10', 'n = 2000, k = 100', 'n = 2000, k = 200',
                              'n = 10000, k = 10', 'n = 10000, k = 100', 'n = 10000, k = 200'])
print(df_s3)

                    Avg error  Std of error  Std among runs
n = 50, k = 10         23.414      5.349636        3.756767
n = 50, k = 100        15.642      9.890897        9.128069
n = 50, k = 200        11.808      9.091267        8.564808
n = 100, k = 10        34.230     20.344559       18.728326
n = 100, k = 100       13.446     15.368509       14.097856
n = 100, k = 200       12.892     13.459433       12.274292
n = 500, k = 10         9.660      8.758790        5.168614
n = 500, k = 100        9.846     10.211282        8.994180
n = 500, k = 200        9.984     10.635212        9.157683
n = 2000, k = 10       34.372     13.569732        2.052767
n = 2000, k = 100       8.440      9.395233        7.694983
n = 2000, k = 200       9.910      9.690712        8.674274
n = 10000, k = 10     204.852     36.755219        0.769329
n = 10000, k = 100     14.766     10.733836        4.406293
n = 10000, k = 200      9.394      8.578273        5.831657
